# Week 2

What is not examined in this notebook is the Boyer-Moore algorithm, which is explored as stand-alone, buildable executables from C and C++. It is neat to see that Boyer-Moore is built into C++17 by default and the C implementation is not too hard either.

What will be used in this notebook is an implementation from the instructors of the course. The nice feature of the module is that we can run its unit tests ourselves!

In [18]:
from pathlib import Path
from typing import Literal, Union
import unittest

from Bio import SeqIO
import numpy as np

from src.bm_preproc import BoyerMoore
from src.kmer_index import Index

In [3]:
!pytest src/bm_preproc.py::TestBoyerMoorePreproc -v

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-7.2.0, pluggy-1.0.0 -- /Users/mhogan/Documents/algorithms-genomic-sequencing/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/mhogan/Documents/algorithms-genomic-sequencing
plugins: anyio-3.6.2
collected 12 items                                                             

src/bm_preproc.py::TestBoyerMoorePreproc::test_big_l_prime_1 PASSED      [  8%]
src/bm_preproc.py::TestBoyerMoorePreproc::test_big_l_prime_2 PASSED      [ 16%]
src/bm_preproc.py::TestBoyerMoorePreproc::test_good_suffix_match_mismatch_1 PASSED [ 25%]
src/bm_preproc.py::TestBoyerMoorePreproc::test_good_suffix_table_1 PASSED [ 33%]
src/bm_preproc.py::TestBoyerMoorePreproc::test_good_suffix_table_2 PASSED [ 41%]
src/bm_preproc.py::TestBoyerMoorePreproc::test_n_1 PASSED                [ 50%]
src/bm_preproc.py::TestBoyerMoorePreproc::test_n_2 PASSED                [ 58%]
src/bm_preproc.py::Test

Neat! All the unit tests passed meaning we do not have to worry about Python2 or Python3 version differences.

In [4]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
!mv chr1.GRCh38.excerpt.fasta week2hw

--2022-11-28 16:15:07--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 108.156.200.29, 108.156.200.25, 108.156.200.204, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|108.156.200.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  --.-KB/s    in 0.1s    

2022-11-28 16:15:08 (5.96 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



## Task

Implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and return (a) the number of character comparisons performed and (b) the number of alignments tried. Roughly speaking, these measure how much work the two different algorithms are doing.

In [5]:
def naive(
    pat: str,
    ref: str,
    full:bool=False
) -> Union[list[int], tuple[list[int], int, int]]:
    """Find all the alignments use naive pattern matching of a pattern
    against a reference string

    Parameters
    ----------
    pat : str
        Pattern string
    ref : str
        Reference string
    full : bool
        Return full comparison (default=False)

    Returns
    -------
    occurrences : list[int]
        Alignment offsets
    alignments : int
        Number of alignment tried. Returned only if `full` is True.
    comparisons : int
        Number of character comparisons performed. Returned only if `full` is True

    """
    comparisons, alignments = 0, 0
    occurrences: list[int] = []
    for i in range(len(ref) - len(pat) + 1):  # loop over alignments
        alignments += 1
        match = True
        for j in range(len(pat)):  # loop over characters
            is_char_match = ref[i + j] == pat[j]
            comparisons += 1
            if not is_char_match:
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    if not full:
        return occurrences
    return occurrences, alignments, comparisons


def boyer_moore(
    pat: str,
    p_bm: BoyerMoore,
    tex: str,
    full: bool = False
) -> Union[list[int], tuple[list[int], int, int]]:
    """Run a pattern search using the Boyer-Moore algorithm

    Parameters
    ----------
    pat : str
        Pattern
    p_bm : BoyerMoore
        Preprocessor for the pattern
    tex : str
        Text to search
    full : bool
        Return full comparison (default=False)

    Returns
    -------
    occurrences : list[int]
        Alignment offsets
    alignments : int
        Number of alignment tried. Returned only if `full` is True.
    comparisons : int
        Number of character comparisons performed. Returned only if `full` is True

    """
    index_i = 0
    comparisons, alignments = 0, 0
    occurrences: list[int] = []
    while index_i < len(tex) - len(pat) + 1:
        alignments += 1
        shift = 1
        mismatched = False
        for index_j in range(len(pat) - 1, -1, -1):
            is_char_match = pat[index_j] == tex[index_i + index_j]
            comparisons += 1
            if not is_char_match:
                skip_bc = p_bm.bad_character_rule(index_j, tex[index_i + index_j])
                skip_gs = p_bm.good_suffix_rule(index_j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(index_i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        index_i += shift
    if not full:
        return occurrences
    return occurrences, alignments, comparisons


In [6]:
class NaiveWithCountsTestCase(unittest.TestCase):
    """Test the occurrences, alignments, and character comparisons of the Naive algorithm"""

    def test_short_patterns(self):
        p_1 = 'word'
        t_1 = 'there would have been a time for such a word'
        naive_results_1 = naive(p_1, t_1, full=True)
        self.assertListEqual(
            list(naive_results_1),
            [[40], 41, 46]
        )

        p_2 = 'needle'
        t_2 = 'needle need noodle needle'
        naive_results_2 = naive(p_2, t_2, full=True)
        self.assertListEqual(
            list(naive_results_2),
            [[0, 19], 20, 35]
        )


class BoyerMooreWithCountsTestCase(unittest.TestCase):
    """Test the occurrences, alignments, and character comparisons of the Boyer-Moore algorithm"""

    def test_short_patterns(self):
        lowercase_alphabet = (
            "".join([chr(index) for index in range(ord("a"), ord("z")+1)])  # letters
            + " "  # empty space
        )

        p_1 = "word"
        t_1 = "there would have been a time for such a word"
        p_bm_1 = BoyerMoore(p_1, lowercase_alphabet)
        bm_results_1 = boyer_moore(p_1, p_bm_1, t_1, full=True)
        self.assertListEqual(
            list(bm_results_1),
            [[40], 12, 15]
        )

        p_2 = "needle"
        t_2 = "needle need noodle needle"
        p_bm_2 = BoyerMoore(p_2, lowercase_alphabet)
        bm_results_2 = boyer_moore(p_2, p_bm_2, t_2, full=True)
        self.assertListEqual(
            list(bm_results_2),
            [[0, 19], 5, 18]
        )


res = unittest.main(argv=[''], verbosity=3, exit=False)
assert len(res.result.failures) == 0

test_short_patterns (__main__.BoyerMooreWithCountsTestCase) ... ok
test_short_patterns (__main__.NaiveWithCountsTestCase) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


Let's now read in the FASTA file with the human Alu sequences

In [7]:
with Path("week2hw/chr1.GRCh38.excerpt.fasta").open(mode="r") as fh:
    alu_sequences: SeqIO.SeqRecord = list(SeqIO.parse(fh, "fasta"))[0]

# Quiz

## Q1
How many alignments does the naive exact matching algorithm try when matching the string
`GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG`
(derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)


First verify that the sequence loaded into memory is from humans (Homo sapiens) and is chromosome 1

In [8]:
print(str(alu_sequences))

ID: CM000663.2_excerpt
Name: CM000663.2_excerpt
Description: CM000663.2_excerpt EXCERPT FROM CM000663.2 Homo sapiens chromosome 1, GRCh38 reference primary assembly
Number of features: 0
Seq('TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAG...AGG')


Now run the query

In [9]:
p_1 = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
_, align_1, _ = naive(p_1, str(alu_sequences.seq), full=True)
print(f"Number of alignments: {align_1}")

Number of alignments: 799954


## Q2:
How many character comparisons does the naive exact matching algorithm try when matching the string
`GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG`
(derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [10]:
p_2 = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
_, _, comp_2 = naive(p_2, str(alu_sequences.seq), full=True)
print(f"Number of comparisons: {comp_2}")

Number of comparisons: 984143


## Q3:
How many alignments does Boyer-Moore try when matching the string
`GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG`
(derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [11]:
p_3 = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
bm_indexer_3 = BoyerMoore(p_3)
_, align_3, _ = boyer_moore(p_3, bm_indexer_3, str(alu_sequences.seq), full=True)
print(f"Number of alignments: {align_3}")

Number of alignments: 127974


## Q4: Index-assisted approximate matching.

In practicals, we built a Python class called Index

implementing an ordered-list version of the k-mer index.  The Index class is copied below

```python
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits
```

We also implemented the pigeonhole principle using Boyer-Moore as our exact matching algorithm.

Implement the pigeonhole principle using Index to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Download the Python module for building a k-mer index.

https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mm function implemented in the previous module.


In [21]:
def exceeds_mismatches(
    pat: str,
    tex: str,
    mat: int,
    start: int,
    end: int,
    mm: int
) -> bool:
    """Verify that the number of mismatches of a hit index of a pattern in text
     does not exceed a fixed value

    Parameters
    ----------
    pat : str
        Pattern
    tex : str
        Text
    mat : int
        Matching index
    start : int
        Start index of text
    end : int
        End index of text
    mm : int
        Number of mismatches allowed

    Returns
    -------
    bool
        False if the number of mismatches is less than input `mm`. True otherwise
    """
    mat_start_index = mat - start
    if (
            mat < start
            or mat_start_index + len(pat) > len(tex)
    ):
        return True

    mismatches = 0

    # Left of text
    for index_j in range(0, start):
        if pat[index_j] != tex[mat_start_index + index_j]:
            mismatches += 1
            if mismatches > mm:
                break

    # Right of pattern
    for index_j in range(end, len(pat)):
        if pat[index_j] != tex[mat_start_index + index_j]:
            mismatches += 1
            if mismatches > mm:
                break

    if mismatches <= mm:
        return False
    return True


def approximate_match_boyermoore(
    pat: str,
    tex: str,
    mm: int,
    alphabet: str = "ACGT"
) -> list[int]:
    """Return the offset occurrences of a pattern against a text using the
     Boyer-Moore algorithm allowing for a fixed number of mismatches

    Parameters
    ----------
    pat : str
        Pattern
    tex : str
        Text
    mm : int
        Number of mismatches allowed
    alphabet : str
        Alphabet of string for processing

    Returns
    -------
    list[int]
        List of occurrences
    """
    segment_len = int(round(len(pat) / (mm + 1)))
    all_matches: set[int] = set()
    for index_i in range(mm + 1):
        start_index = index_i * segment_len
        end_index = min((index_i + 1) * segment_len, len(pat))
        bm_pat = pat[start_index: end_index]
        bm_indexer = BoyerMoore(bm_pat, alphabet)
        bm_matches = boyer_moore(bm_pat, bm_indexer, tex)
        mat: int
        for mat in bm_matches:
            if not exceeds_mismatches(pat, tex, mat, start_index, end_index, mm):
                mat_start_index = mat - start_index
                all_matches.add(mat_start_index)
    return list(all_matches)


def approximate_match_kmer(
    pat: str,
    tex: str,
    mm: int
) -> list[int]:
    """Return the offset occurrences of a pattern against a text using the
     pigeon-hole principle with k-mers allowing for a fixed number of mismatches.
     The number of k-mers is one more than the number of mismatches allowed

    Parameters
    ----------
    pat : str
        Pattern
    tex : str
        Text
    mm : int
        Number of mismatches allowed

    Returns
    -------
    list[int]
        List of occurrences
    """
    kmer_len = int(round(len(pat) / (mm + 1)))
    kmer_indexer = Index(tex, kmer_len)
    all_matches: set[int] = set()
    for index_i in range(mm + 1):
        start_index = index_i * kmer_len
        end_index = min((index_i + 1) * kmer_len, len(pat))
        kmer_pat = pat[start_index: end_index]
        kmer_matches = kmer_indexer.query(kmer_pat)
        # print("Number of k-mer ")
        mat: int
        for mat in kmer_matches:
            if not exceeds_mismatches(pat, tex, mat, start_index, end_index, mm):
                mat_start_index = mat - start_index
                all_matches.add(mat_start_index)
    return list(all_matches)


def approximate_match(
    pat: str,
    tex: str,
    mm: int,
    method: Literal["BoyerMoore", "Index"],
    alphabet: str = "ACGT",
) -> list[int]:
    """Return the offset occurrences of a pattern against a text using
    either the Boyer-Moore algorithm or k-mer indexing.

    Parameters
    ----------
    pat : str
        Pattern
    tex : str
        Text
    mm : int
        Number of mismatches allowed
    method : Literal["BoyerMoore", "Index"]
        Method to use
    alphabet : str
        Alphabet, required when the method is Boyer-Moore

    Returns
    -------
    list[int]
        List of occurrences
    """
    if method == "BoyerMoore":
        return approximate_match_boyermoore(pat, tex, mm, alphabet)
    if method == "Index":
        return approximate_match_kmer(pat, tex, mm)
    raise ValueError(f"Method {method} is not supported")


In [22]:
class ApproximateMatchBoyerMooreTestCase(unittest.TestCase):

    def test_short_patterns(self):
        pat = "AACTTG"
        tex = "CACTTAATTTG"
        bm_1 = approximate_match(pat, tex, 1, "BoyerMoore")
        self.assertListEqual(bm_1, [5])

        bm_2 = approximate_match(pat, tex, 2, "BoyerMoore")
        self.assertListEqual(bm_2, [0, 5])


class ApproximateMatchKmerTestCase(unittest.TestCase):

    def test_short_patterns(self):
        pat = "AACTTG"
        tex = "CACTTAATTTG"
        bm_1 = approximate_match(pat, tex, 1, "Index")
        self.assertListEqual(bm_1, [5])

        bm_2 = approximate_match(pat, tex, 2, "Index")
        self.assertListEqual(bm_2, [0, 5])


res = unittest.main(argv=[''], verbosity=3, exit=False)
assert len(res.result.failures) == 0

test_short_patterns (__main__.ApproximateMatchBoyerMooreTestCase) ... ok
test_short_patterns (__main__.ApproximateMatchKmerTestCase) ... ok
test_short_patterns (__main__.BoyerMooreWithCountsTestCase) ... ok
test_short_patterns (__main__.NaiveWithCountsTestCase) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.043s

OK


Now I can try to finally answer the question

```
How many times does the string GGCGCGGTGGCTCACGCCTGTAAT,
which is derived from a human Alu sequence, occur with
up to 2 substitutions in the excerpt of human chromosome
1?  (Don't consider reverse complements here.)
```

In [24]:
p_4 = "GGCGCGGTGGCTCACGCCTGTAAT"
occurrences_4 = approximate_match(
    p_4, str(alu_sequences.seq), 2, "Index"
)
print(f"Number of occurrences: {len(occurrences_4)}")

Number of occurrences: 19
